In [1]:
# from cenpy import products
import matplotlib.pyplot as plt
import cenpy
import pandas as pd 
import os
import numpy as np
pd.set_option('display.max_columns', None)
import geopandas as gpd

/home/teamlary/miniconda3/envs/hospitalization/lib/python3.9/site-packages/libpysal/cg/alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
/home/teamlary/miniconda3/envs/hospitalization/lib/python3.9/site-packages/libpysal/cg/alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_faces(triangle):
/home/teamlar

In [15]:
zip_2010 = "D:/GitHub/merraData/tx_texas_zip_codes_geo.min.json"
zip_2010 = "../assets/tx_texas_zip_codes_geo.min.json"
tx_zip = gpd.read_file(zip_2010)
tx_zip = tx_zip.rename(columns={
    'ZCTA5CE10': 'PAT_ZIP'
})
tx_zip

,STATEFP10,PAT_ZIP,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,48,75801,4875801,B5,G6350,S,555807428,6484251,+31.7345202,-095.5313809,N,"POLYGON ((-95.68072 31.72800, -95.68085 31.728..."
1,48,75839,4875839,B5,G6350,S,381863557,3928555,+31.6066930,-095.5833403,N,"POLYGON ((-95.66269 31.64586, -95.66242 31.645..."
2,48,78336,4878336,B5,G6350,S,126923194,31624523,+27.9269126,-097.1777757,N,"POLYGON ((-97.19642 27.91194, -97.19618 27.912..."
3,48,76389,4876389,B5,G6350,S,515265783,10157866,+33.5126278,-098.4576268,N,"MULTIPOLYGON (((-98.37035 33.44176, -98.37820 ..."
4,48,76310,4876310,B5,G6350,S,465637930,24888388,+33.7990847,-098.5098400,N,"POLYGON ((-98.50723 33.84419, -98.50235 33.844..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1934,48,79788,4879788,B5,G6350,S,11978926,14255,+31.5824731,-102.9768502,N,"POLYGON ((-103.01745 31.56573, -103.01751 31.5..."
1935,48,76364,4876364,B5,G6350,S,177003889,419459,+34.0428171,-098.9929523,N,"MULTIPOLYGON (((-99.02675 34.13538, -99.03759 ..."
1936,48,78598,4878598,B5,G6350,S,14624694,3796082,+26.5693082,-097.4423313,N,"POLYGON ((-97.43430 26.59022, -97.43398 26.589..."
1937,48,78665,4878665,B5,G6350,S,48574015,774191,+30.5433562,-097.6455901,N,"POLYGON ((-97.63808 30.57590, -97.63804 30.575..."


### Age, sex

DEC
- P008002 - P008040 (Male from Total male to 85 years and over)
- P008041 - P008079 (Female from Total female to 85 years and over)
Need to sum columns to reflect the grouping done by ACS

ACS
- B01001_002E - B01001_025E (Male from Total male to 85 years and over)
- B01001_026E - B01001_049E (Female from Total female to 85 years and over)

### Race

DEC
- P006001 - P006008 (Total populations by race (white, black, native, asian, pacific, other, two or more))

ACS 
- B02001_001E - B02001_008E (Total populations by race (white, black, native, asian, pacific, other, two or more))

### Income/Poverty

DEC
- P152A - P152H (Median Household Income, by race - white, black, native, asian, pacific, other, two or more, not hispanic, hispanic)
- P159A002 - P159H002 (Poverty status, by race )

ACS 
- B19013A_001E - B19013H_001E (median house hold income, by race)
- B17020A_002E - B17020H_002E (poverty status, by race)

### Education

DEC
- P037001 - P037005 (Sex by educational attainment for population 25 years and over)
- B15002_001E - B15035_001E (Sex by educational attainment for population 25 years and over)


---

- A    White alone
- B    Black or African American Alone
- C    American Indian and Alaska Native Alone
- D    Asian Alone
- E    Native Hawaiian and Other Pacific Islander Alone
- F    Some Other Race Alone
- G    Two or More Races
- H    White Alone, Not Hispanic or Latino
- I    Hispanic or Latino 

In [6]:
survey = 'DECENNIALSF32000'
# survey = 'ACSDT5Y2011'
dec_conn = cenpy.remote.APIConnection(survey)
table_name = ''
if 'DEC' in survey:
    table_name = 'Census 2000 SF3 Table Number'
elif 'ACS' in survey:
    table_name = 'ACS Table Number'
    # survey_vars = conn.variables[conn.variables['group'].isin(identical[table_name])

# survey_vars = conn.variables[conn.variables['group'].isin(final_variables['Census 2000 SF3 Table Number'].values)]

In [5]:
def switch_last_and_second_to_last(lst):
    if len(lst) < 2:
        # Handle the case where the list has less than 2 elements
        return lst

    # Swap the last and second-to-last elements
    lst[-1], lst[-2] = lst[-2], lst[-1]

In [14]:
def switch_last_2(lst):
    if len(lst) < 2:
        # Handle the case where the list has less than 2 elements
        return lst

    # Swap the last and second-to-last elements
    # lst[-1], lst[-2] = lst[-2], lst[-1]
    lst[-4], lst[-3], lst[-2], lst[-1] = lst[-2], lst[-1], lst[-4], lst[-3]

In [12]:
switch_last_and_second_to_last(dec_vars)

In [8]:
base_var_name = 'P008'
dec_vars = sorted([i for i in dec_conn.variables.index if i.startswith(base_var_name) ]) #and (i.endswith('002') or i.endswith('001'))])
# dec_vars = dec_vars
# dec_vars = dec_vars[1:18]
dec_vars = dec_vars
# switch_last_2(dec_vars)
dec_vars

['P008001',
 'P008002',
 'P008003',
 'P008004',
 'P008005',
 'P008006',
 'P008007',
 'P008008',
 'P008009',
 'P008010',
 'P008011',
 'P008012',
 'P008013',
 'P008014',
 'P008015',
 'P008016',
 'P008017',
 'P008018',
 'P008019',
 'P008020',
 'P008021',
 'P008022',
 'P008023',
 'P008024',
 'P008025',
 'P008026',
 'P008027',
 'P008028',
 'P008029',
 'P008030',
 'P008031',
 'P008032',
 'P008033',
 'P008034',
 'P008035',
 'P008036',
 'P008037',
 'P008038',
 'P008039',
 'P008040',
 'P008041',
 'P008042',
 'P008043',
 'P008044',
 'P008045',
 'P008046',
 'P008047',
 'P008048',
 'P008049',
 'P008050',
 'P008051',
 'P008052',
 'P008053',
 'P008054',
 'P008055',
 'P008056',
 'P008057',
 'P008058',
 'P008059',
 'P008060',
 'P008061',
 'P008062',
 'P008063',
 'P008064',
 'P008065',
 'P008066',
 'P008067',
 'P008068',
 'P008069',
 'P008070',
 'P008071',
 'P008072',
 'P008073',
 'P008074',
 'P008075',
 'P008076',
 'P008077',
 'P008078',
 'P008079']

In [9]:
dec_acs_dict = dict(zip(dec_vars, acs_vars))
dec_acs_dict = dict(zip(df.columns, acs_vars))
dec_acs_dict

NameError: name 'df' is not defined

In [ ]:
dec_groups = ['P006','P152','P159', 'P037 '] # race, income, poverty, education

In [20]:
def getDecennialData(dec_vars):
    data = dec_conn.query(
        # ['B01003_001E', 'B01001_001E', 'B01001_002E', 'B01001_026E'],
        # ['P001001','P053001','H011011','P037009','P037010','P037011','P065001'],
        # dec_vars,
        dec_vars,
                geo_unit="zip code tabulation area"
    )

    df = pd.DataFrame(data)

    if 'state' in df.columns:
        df = df[df['state'] == '48']
    df.sort_values('zip code tabulation area')
    df.rename(columns={
        'zip code tabulation area': 'PAT_ZIP'
    })
    base_df = df.copy()

    return base_df

# survey = 'DECENNIALSF32000'
def getACSvariables(race):
    survey = 'ACSDT5Y2011'
    acs_conn = cenpy.remote.APIConnection(survey)
    table_name = ''
    if 'DEC' in survey:
        table_name = 'Census 2000 SF3 Table Number'
    elif 'ACS' in survey:
        table_name = 'ACS Table Number'
        # survey_vars = conn.variables[conn.variables['group'].isin(identical[table_name])
    acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith(f'B01001{race}') ])#and (i.endswith('002E') or i.endswith('001E'))])
    # survey_vars = conn.variables[conn.variables['group'].isin(final_variables['Census 2000 SF3 Table Number'].values)]
    # acs_vars = acs_conn.variables.loc[acs_conn.variables.index.str.startswith('B02001')].sort_index().index
    # acs_vars = acs_vars[:-1]
    return acs_vars

In [92]:
data = dec_conn.query(
    # ['B01003_001E', 'B01001_001E', 'B01001_002E', 'B01001_026E'],
    # ['P001001','P053001','H011011','P037009','P037010','P037011','P065001'],
    # dec_vars,
    dec_vars,
            geo_unit="zip code tabulation area"
)

df = pd.DataFrame(data)

if 'state' in df.columns:
    df = df[df['state'] == '48']
df.sort_values('zip code tabulation area')
df.rename(columns={
    'zip code tabulation area': 'PAT_ZIP'
})
base_df = df.copy()

In [133]:
P145_vars = [f'P145A{i:03}' for i in range(1,50)]
male_anchor = 1
m_20_24 = P145_vars[male_anchor+6:male_anchor+6+3]
m_35_44 = P145_vars[male_anchor+11:male_anchor+11+2]
m_45_54 = P145_vars[male_anchor+13:male_anchor+13+2]
m_55_64 = P145_vars[male_anchor+15:male_anchor+15+3]
m_65_74 = P145_vars[male_anchor+18:male_anchor+18+3]
m_75_84 = P145_vars[male_anchor+21:male_anchor+21+2]

female_anchor = 25
f_20_24 = P145_vars[female_anchor+6:female_anchor+6+3]
f_35_44 = P145_vars[female_anchor+11:female_anchor+11+2]
f_45_54 = P145_vars[female_anchor+13:female_anchor+13+2]
f_55_64 = P145_vars[female_anchor+15:female_anchor+15+3]
f_65_74 = P145_vars[female_anchor+18:female_anchor+18+3]
f_75_84 = P145_vars[female_anchor+21:female_anchor+21+2]

# thirtyfive_fortyfour = ['P145013','P145014']
# fortyfive_fiftyfour = ['P145015','P145016']
# fiftyfive_sixyfour = ['P14517','']

In [104]:
P145_vars[2:7]+P145_vars[10:12]+P145_vars[24:25]

['P145A003',
 'P145A004',
 'P145A005',
 'P145A006',
 'P145A007',
 'P145A011',
 'P145A012',
 'P145A025']

In [147]:
derived_male_df = pd.DataFrame({
    f'{base_var_name}008': df[m_20_24].astype(int).sum(axis=1),
    f'{base_var_name}011': df[m_35_44].astype(int).sum(axis=1),
    f'{base_var_name}012': df[m_45_54].astype(int).sum(axis=1),
    f'{base_var_name}013': df[m_55_64].astype(int).sum(axis=1),
    f'{base_var_name}014': df[m_65_74].astype(int).sum(axis=1),
    f'{base_var_name}015': df[m_75_84].astype(int).sum(axis=1),
})
male_cols_to_keep = df.loc[:,P145_vars[:7]+P145_vars[10:12]+P145_vars[24:25]]
male_cols_to_keep = male_cols_to_keep.rename(columns={
    f'{base_var_name}011': f'{base_var_name}009',
    f'{base_var_name}012': f'{base_var_name}010',
    f'{base_var_name}025': f'{base_var_name}016',
    
})

derived_female_df = pd.DataFrame({
    f'{base_var_name}023': df[f_20_24].astype(int).sum(axis=1),
    f'{base_var_name}026': df[m_35_44].astype(int).sum(axis=1),
    f'{base_var_name}027': df[m_45_54].astype(int).sum(axis=1),
    f'{base_var_name}028': df[m_55_64].astype(int).sum(axis=1),
    f'{base_var_name}029': df[m_65_74].astype(int).sum(axis=1),
    f'{base_var_name}030': df[m_75_84].astype(int).sum(axis=1),
})

female_cols_to_keep = df.loc[:,P145_vars[25:31] + P145_vars[34:36]+P145_vars[-1:]]
female_cols_to_keep = female_cols_to_keep.rename(columns=dict(zip(P145_vars[25:31],[f'{base_var_name}{i:03}' for i in range(17,23)])))
female_cols_to_keep = female_cols_to_keep.rename(columns={
    f'{base_var_name}035': f'{base_var_name}024',
    f'{base_var_name}036': f'{base_var_name}025',
    f'{base_var_name}049': f'{base_var_name}031',
})
female_cols_to_keep
derived_df = pd.concat([derived_male_df, derived_female_df, male_cols_to_keep, female_cols_to_keep], axis=1).sort_index(axis=1)
derived_df.insert(0, 'PAT_ZIP', base_df['zip code tabulation area'])

In [157]:
# survey = 'DECENNIALSF32000'
survey = 'ACSDT5Y2011'
acs_conn = cenpy.remote.APIConnection(survey)
table_name = ''
if 'DEC' in survey:
    table_name = 'Census 2000 SF3 Table Number'
elif 'ACS' in survey:
    table_name = 'ACS Table Number'
    # survey_vars = conn.variables[conn.variables['group'].isin(identical[table_name])
acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith('B17020') and (i.endswith('002E') or i.endswith('001E'))])
acs_vars = acs_vars[:2]
print(acs_vars)
# survey_vars = conn.variables[conn.variables['group'].isin(final_variables['Census 2000 SF3 Table Number'].values)]
# acs_vars = acs_conn.variables.loc[acs_conn.variables.index.str.startswith('B02001')].sort_index().index
data = acs_conn.query(
    # ['B01003_001E', 'B01001_001E', 'B01001_002E', 'B01001_026E'],
    # ['P001001','P053001','H011011','P037009','P037010','P037011','P065001'],
    ['B02001_001E','B02001_002E'] + acs_vars,
            geo_unit="zip code tabulation area"
)

df = pd.DataFrame(data)

if 'state' in df.columns:
    df = df[df['state'] == '48']
df.sort_values('zip code tabulation area')

['B17020A_001E', 'B17020A_002E']


,B02001_001E,B02001_002E,B17020A_001E,B17020A_002E,state,zip code tabulation area
25500,12510,9170,9169,1011,48,75001
25501,62175,46812,46480,1596,48,75002
25502,45931,35798,35629,5307,48,75006
25503,50921,35589,35482,1500,48,75007
25504,8418,7387,7339,374,48,75009
...,...,...,...,...,...,...
28503,17995,12349,12279,489,48,79934
28504,18249,14543,14388,3081,48,79935
28505,114941,96682,96337,14746,48,79936
28506,51367,42417,41155,8578,48,79938


In [31]:
def mergeZipCode(df_census, label_dict, survey, year, code=None):
    print(list(label_dict.items())[0][1].split('_')[0])
    df_census = df_census.rename(columns={
        'zip code tabulation area': 'PAT_ZIP'
    })
    # print(df_census.columns)
    df_census = df_census.rename(columns=label_dict)
    # print(df_census.columns)

    df_census = df_census.merge(tx_zip['PAT_ZIP'], on='PAT_ZIP')
    if 'state' in df_census.columns:
        df_census = df_census.drop('state',axis=1)
    # df_census.to_csv(f'../assets/census/{survey}_{year}_{list(label_dict.items())[0][1].split("_")[0]}.csv', index=False)
    # df_census.to_csv(f'../assets/census/{survey}_{year}_B01001H.csv', index=False)
    print(df_census.columns)
    for column in df_census.columns[:3]:
        df_census.loc[:,['PAT_ZIP']+[column]].to_csv(f'../assets/census/{survey}_{year}_{column}.csv', index=False)
    # df_census.to_csv(f'../assets/census/{survey}_{year}_B01001.csv', index=False)
    return df_census

In [21]:
dec_pop_vars = sorted([i for i in dec_conn.variables.index if i in ['P008001','P008002','P008041'] ])
temp_df = getDecennialData(dec_pop_vars)
    

In [24]:
pop_dict = {
    'P008001': 'B01001_001E',
    'P008002': 'B01001_002E',
    'P008041': 'B01001_026E'
}

In [32]:
mergeZipCode(temp_df, pop_dict, 'DEC','2000')

B01001
Index(['B01001_001E', 'B01001_002E', 'B01001_026E', 'PAT_ZIP'], dtype='object')


,B01001_001E,B01001_002E,B01001_026E,PAT_ZIP
0,1866,977,889,73949
1,8175,4112,4063,75001
2,37171,18614,18557,75002
3,45990,23046,22944,75006
4,50357,24726,25631,75007
...,...,...,...,...
1831,92136,44312,47824,79936
1832,18379,10396,7983,79938
1833,10274,4782,5492,88063
1834,33185,16034,17151,88220


In [152]:
letters_list = ['A','B','C','D','E','F','G']
age_var_list = [f'P145{i}' for i in letters_list]
age_var_list

['P145A', 'P145B', 'P145C', 'P145D', 'P145E', 'P145F', 'P145G']

In [23]:
dec_acs_dict

{'P008001': 'B01001_001E',
 'P008002': 'B01001_002E',
 'P008003': 'B01001_003E',
 'P008004': 'B01001_004E',
 'P008005': 'B01001_005E',
 'P008006': 'B01001_006E',
 'P008007': 'B01001_007E',
 'P008008': 'B01001_008E',
 'P008009': 'B01001_009E',
 'P008010': 'B01001_010E',
 'P008011': 'B01001_011E',
 'P008012': 'B01001_012E',
 'P008013': 'B01001_013E',
 'P008014': 'B01001_014E',
 'P008015': 'B01001_015E',
 'P008016': 'B01001_016E',
 'P008017': 'B01001_017E',
 'P008018': 'B01001_018E',
 'P008019': 'B01001_019E',
 'P008020': 'B01001_020E',
 'P008021': 'B01001_021E',
 'P008022': 'B01001_022E',
 'P008023': 'B01001_023E',
 'P008024': 'B01001_024E',
 'P008025': 'B01001_025E',
 'P008026': 'B01001_026E',
 'P008027': 'B01001_027E',
 'P008028': 'B01001_028E',
 'P008029': 'B01001_029E',
 'P008030': 'B01001_030E',
 'P008031': 'B01001_031E',
 'P008032': 'B01001_032E',
 'P008033': 'B01001_033E',
 'P008034': 'B01001_034E',
 'P008035': 'B01001_035E',
 'P008036': 'B01001_036E',
 'P008037': 'B01001_037E',
 

In [166]:
def getPopulationByAge(base_var_name):
    dec_vars = sorted([i for i in dec_conn.variables.index if i.startswith(base_var_name) ])
    
    df = getDecennialData(dec_vars)
    acs_vars = getACSvariables(base_var_name[-1])

    dec_acs_dict = dict(zip(df.columns, acs_vars))

    P145_vars = [f'{base_var_name}{i:03}' for i in range(1,50)]
    male_anchor = 1
    m_20_24 = P145_vars[male_anchor+6:male_anchor+6+3]
    m_35_44 = P145_vars[male_anchor+11:male_anchor+11+2]
    m_45_54 = P145_vars[male_anchor+13:male_anchor+13+2]
    m_55_64 = P145_vars[male_anchor+15:male_anchor+15+3]
    m_65_74 = P145_vars[male_anchor+18:male_anchor+18+3]
    m_75_84 = P145_vars[male_anchor+21:male_anchor+21+2]

    female_anchor = 25
    f_20_24 = P145_vars[female_anchor+6:female_anchor+6+3]
    f_35_44 = P145_vars[female_anchor+11:female_anchor+11+2]
    f_45_54 = P145_vars[female_anchor+13:female_anchor+13+2]
    f_55_64 = P145_vars[female_anchor+15:female_anchor+15+3]
    f_65_74 = P145_vars[female_anchor+18:female_anchor+18+3]
    f_75_84 = P145_vars[female_anchor+21:female_anchor+21+2]

    derived_male_df = pd.DataFrame({
    f'{base_var_name}008': df[m_20_24].astype(int).sum(axis=1),
    f'{base_var_name}011': df[m_35_44].astype(int).sum(axis=1),
    f'{base_var_name}012': df[m_45_54].astype(int).sum(axis=1),
    f'{base_var_name}013': df[m_55_64].astype(int).sum(axis=1),
    f'{base_var_name}014': df[m_65_74].astype(int).sum(axis=1),
    f'{base_var_name}015': df[m_75_84].astype(int).sum(axis=1),
})
    male_cols_to_keep = df.loc[:,P145_vars[:7]+P145_vars[10:12]+P145_vars[24:25]]
    male_cols_to_keep = male_cols_to_keep.rename(columns={
        f'{base_var_name}011': f'{base_var_name}009',
        f'{base_var_name}012': f'{base_var_name}010',
        f'{base_var_name}025': f'{base_var_name}016',
        
    })

    derived_female_df = pd.DataFrame({
        f'{base_var_name}023': df[f_20_24].astype(int).sum(axis=1),
        f'{base_var_name}026': df[m_35_44].astype(int).sum(axis=1),
        f'{base_var_name}027': df[m_45_54].astype(int).sum(axis=1),
        f'{base_var_name}028': df[m_55_64].astype(int).sum(axis=1),
        f'{base_var_name}029': df[m_65_74].astype(int).sum(axis=1),
        f'{base_var_name}030': df[m_75_84].astype(int).sum(axis=1),
    })

    female_cols_to_keep = df.loc[:,P145_vars[25:31] + P145_vars[34:36]+P145_vars[-1:]]
    female_cols_to_keep = female_cols_to_keep.rename(columns=dict(zip(P145_vars[25:31],[f'{base_var_name}{i:03}' for i in range(17,23)])))
    female_cols_to_keep = female_cols_to_keep.rename(columns={
        f'{base_var_name}035': f'{base_var_name}024',
        f'{base_var_name}036': f'{base_var_name}025',
        f'{base_var_name}049': f'{base_var_name}031',
    })
    female_cols_to_keep
    derived_df = pd.concat([derived_male_df, derived_female_df, male_cols_to_keep, female_cols_to_keep], axis=1).sort_index(axis=1)
    derived_df.insert(0, 'PAT_ZIP', df['zip code tabulation area'])

    return dec_acs_dict, derived_df


In [185]:
for i in ['P145H']:
    dec_acs_dict, derived_df = getPopulationByAge(i)
    mergeZipCode(derived_df, dec_acs_dict, 'DEC','2000',f'{acs_vars[0].split("_")[0]}')

B01001H


In [178]:
df[df['zip code tabulation area'] == '73949']

,P152A001,P152B001,P152C001,P152D001,P152E001,P152F001,P152G001,P152I001,P152H001,zip code tabulation area
25630,34583,0,102264,35000,0,25625,17917,33988,27813,73949


In [12]:
def acs_survey(variable, year):
    survey = survey = f'ACSDT5Y{year}'
    acs_conn = cenpy.remote.APIConnection(survey)
    acs_vars = []
    if variable == 'B02001':
        acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith(variable)])
        acs_vars = acs_vars[:-2]
    elif variable == 'B19013':
        acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith(variable)])
        acs_vars = acs_vars[:-1]
    elif variable == 'B17020':
        acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith(variable) and (i.endswith('002E') or i.endswith('001E'))])
        # acs_vars = acs_vars
    elif variable == 'B15002':
        acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith(variable)])
        acs_vars = acs_vars[1:]
    elif variable == 'B01001':
        letters_list = ['A','B','C','D','E','F','G','H','I']
        letters_list = ['_001E','_002E','_026E']
        for letter in letters_list:
            acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith(variable+letter)])
            data = acs_conn.query(
                    acs_vars,
                    geo_unit = "zip code tabulation area"
                )

            df = pd.DataFrame(data)

            if 'state' in df.columns:
                df = df[df['state'] == '48']
            df = df.sort_values('zip code tabulation area')
            df = df.rename(columns={
                'zip code tabulation area': 'PAT_ZIP'
            })
            
            df = df.merge(tx_zip['PAT_ZIP'], on='PAT_ZIP')

            if 'state' in df.columns:
                df = df.drop('state',axis=1)


            df = df[['PAT_ZIP'] + [col for col in df.columns if col != 'PAT_ZIP']]
            
            df.to_csv(f'../assets/census/ACS5Y_{year}_{variable+letter}.csv', index=False)
        
    data = acs_conn.query(
        acs_vars,
        geo_unit = "zip code tabulation area"
    )

    df = pd.DataFrame(data)

    if 'state' in df.columns:
        df = df[df['state'] == '48']
    df = df.sort_values('zip code tabulation area')
    df = df.rename(columns={
        'zip code tabulation area': 'PAT_ZIP'
    })
    
    df = df.merge(tx_zip['PAT_ZIP'], on='PAT_ZIP')

    if 'state' in df.columns:
        df = df.drop('state',axis=1)


    df = df[['PAT_ZIP'] + [col for col in df.columns if col != 'PAT_ZIP']]
    
    df.to_csv(f'../assets/census/ACS5Y_{year}_{variable}.csv', index=False)


In [16]:
variable_list = ['B02001','B19013','B17020','B15002']
variable_list = ['B01001']
year_list = ['2011','2016','2021']
for year in year_list:
    for variable in variable_list:
        print(year, variable)
        acs_survey(variable, year)

2011 B01001
2016 B01001
2021 B01001


---
### Separating ACS2011, 2016, 2021 B15002 variables to male and female

In [29]:
file_path = '../assets/census/customVariables/'
census_files = [file_path+i for i in os.listdir(file_path) if 'ACS' in i and 'B15002' in i]
census_files

['../assets/census/customVariables/ACS5Y_2011_B15002.csv',
 '../assets/census/customVariables/ACS5Y_2016_B15002.csv',
 '../assets/census/customVariables/ACS5Y_2021_B15002.csv']

In [45]:
male_range = [f'B15002_{i:03}E' for i in range(2,19)]
female_range = [f'B15002_{i:03}E' for i in range(19,36)]
for file in census_files:
    print(file[:-4])
    df = pd.read_csv(file)
    male_df = df.loc[:, ['PAT_ZIP'] + male_range]
    male_df.to_csv(f'{file[:-4]}M.csv', index=False)

    female_df = df.loc[:, ['PAT_ZIP'] + female_range]
    female_df.to_csv(f'{file[:-4]}F.csv', index=False)

    # df.loc[:,['PAT_ZIP'] +male_range]

../assets/census/customVariables/ACS5Y_2011_B15002
../assets/census/customVariables/ACS5Y_2016_B15002
../assets/census/customVariables/ACS5Y_2021_B15002


---

### Test ACS variables

In [9]:
# survey = 'DECENNIALSF32000'
survey = 'ACSDT5Y2011'
acs_conn = cenpy.remote.APIConnection(survey)
table_name = ''
if 'DEC' in survey:
    table_name = 'Census 2000 SF3 Table Number'
elif 'ACS' in survey:
    table_name = 'ACS Table Number'
    # survey_vars = conn.variables[conn.variables['group'].isin(identical[table_name])

# survey_vars = conn.variables[conn.variables['group'].isin(final_variables['Census 2000 SF3 Table Number'].values)]
# acs_vars = acs_conn.variables.loc[acs_conn.variables.index.str.startswith('B02001')].sort_index().index

In [28]:
acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith('B15002') ])# and i.endswith('002E')])
acs_vars

['B15002_001E',
 'B15002_002E',
 'B15002_003E',
 'B15002_004E',
 'B15002_005E',
 'B15002_006E',
 'B15002_007E',
 'B15002_008E',
 'B15002_009E',
 'B15002_010E',
 'B15002_011E',
 'B15002_012E',
 'B15002_013E',
 'B15002_014E',
 'B15002_015E',
 'B15002_016E',
 'B15002_017E',
 'B15002_018E',
 'B15002_019E',
 'B15002_020E',
 'B15002_021E',
 'B15002_022E',
 'B15002_023E',
 'B15002_024E',
 'B15002_025E',
 'B15002_026E',
 'B15002_027E',
 'B15002_028E',
 'B15002_029E',
 'B15002_030E',
 'B15002_031E',
 'B15002_032E',
 'B15002_033E',
 'B15002_034E',
 'B15002_035E']

In [14]:
acs_vars_full = []

In [27]:
acs_vars_full.append(acs_vars)
acs_vars_full

[['B02001_001E',
  'B02001_002E',
  'B02001_003E',
  'B02001_004E',
  'B02001_005E',
  'B02001_006E',
  'B02001_007E',
  'B02001_008E'],
 ['B19013A_001E',
  'B19013B_001E',
  'B19013C_001E',
  'B19013D_001E',
  'B19013E_001E',
  'B19013F_001E',
  'B19013G_001E',
  'B19013H_001E',
  'B19013I_001E'],
 ['B17020A_002E',
  'B17020B_002E',
  'B17020C_002E',
  'B17020D_002E',
  'B17020E_002E',
  'B17020F_002E',
  'B17020G_002E',
  'B17020H_002E',
  'B17020I_002E']]

In [36]:
# survey = 'DECENNIALSF32000'
survey = 'ACSDT5Y2011'
acs_conn = cenpy.remote.APIConnection(survey)
table_name = ''
if 'DEC' in survey:
    table_name = 'Census 2000 SF3 Table Number'
elif 'ACS' in survey:
    table_name = 'ACS Table Number'
    # survey_vars = conn.variables[conn.variables['group'].isin(identical[table_name])
acs_vars = sorted([i for i in acs_conn.variables.index if i.startswith('B15002') ])# and i.endswith('002E')])
# survey_vars = conn.variables[conn.variables['group'].isin(final_variables['Census 2000 SF3 Table Number'].values)]
# acs_vars = acs_conn.variables.loc[acs_conn.variables.index.str.startswith('B02001')].sort_index().index
data = acs_conn.query(
    # ['B01003_001E', 'B01001_001E', 'B01001_002E', 'B01001_026E'],
    # ['P001001','P053001','H011011','P037009','P037010','P037011','P065001'],
    # dec_vars,
    acs_vars,
            geo_unit="zip code tabulation area"
)

df = pd.DataFrame(data)

if 'state' in df.columns:
    df = df[df['state'] == '48']
df.sort_values('zip code tabulation area')

,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B02001_009E,B02001_010E,state,zip code tabulation area
25500,12510,9170,1444,34,686,136,891,149,38,111,48,75001
25501,62175,46812,6156,90,5272,0,1470,2375,678,1697,48,75002
25502,45931,35798,3461,328,3206,0,2525,613,152,461,48,75006
25503,50921,35589,4542,593,6820,30,1634,1713,357,1356,48,75007
25504,8418,7387,112,207,0,0,505,207,136,71,48,75009
...,...,...,...,...,...,...,...,...,...,...,...,...
28503,17995,12349,2694,9,332,16,1481,1114,451,663,48,79934
28504,18249,14543,584,89,98,22,2527,386,206,180,48,79935
28505,114941,96682,2237,576,737,0,12809,1900,1403,497,48,79936
28506,51367,42417,1381,437,401,72,5320,1339,717,622,48,79938


In [31]:
data = conn.query(
    # ['B01003_001E', 'B01001_001E', 'B01001_002E', 'B01001_026E'],
    # ['P001001','P053001','H011011','P037009','P037010','P037011','P065001'],
    # dec_vars,
    dec_vars_full[0],
            geo_unit="zip code tabulation area"
)

df = pd.DataFrame(data)

if 'state' in df.columns:
    df = df[df['state'] == '48']
df.sort_values('zip code tabulation area')

,P006001,P006002,P006003,P006004,P006005,P006006,P006007,P006008,zip code tabulation area
0,19143,18187,161,89,0,0,470,236,00601
1,42042,37309,1334,11,37,0,2129,1222,00602
2,55592,46272,2989,11,46,0,4780,1494,00603
3,3844,3215,226,0,49,0,261,93,00604
4,6449,5821,175,5,0,0,200,248,00606
...,...,...,...,...,...,...,...,...,...
33173,136,132,0,3,0,0,0,1,99927
33174,2424,1749,0,376,26,0,0,273,99929
33175,47,47,0,0,0,0,0,0,99950
33176,0,0,0,0,0,0,0,0,999HH
